In [17]:
import psycopg2

import json

import os

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [6]:
connection.rollback()

query = """

select distinct cu.zip
from customers cu
order by 1

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

zip_list = []

for row in rows:
    zip_list.append(row[0])

print('Number of zip codes:',  len(zip_list))
print(zip_list)


Number of zip codes: 550
['33004', '33009', '33010', '33012', '33013', '33014', '33015', '33016', '33018', '33019', '33020', '33021', '33023', '33024', '33025', '33026', '33027', '33028', '33029', '33032', '33033', '33039', '33054', '33055', '33056', '33109', '33122', '33125', '33126', '33127', '33128', '33129', '33130', '33131', '33132', '33133', '33134', '33135', '33136', '33137', '33138', '33139', '33140', '33141', '33142', '33143', '33144', '33145', '33146', '33147', '33149', '33150', '33154', '33155', '33156', '33157', '33158', '33160', '33161', '33162', '33165', '33166', '33167', '33168', '33169', '33170', '33172', '33173', '33174', '33175', '33176', '33177', '33178', '33179', '33180', '33181', '33182', '33183', '33184', '33185', '33186', '33187', '33189', '33190', '33193', '33196', '33301', '33312', '33314', '33315', '33316', '33317', '33324', '33325', '33328', '33330', '33331', '33332', '37013', '37014', '37015', '37027', '37035', '37048', '37062', '37064', '37066', '37067', '3

In [15]:
connection.rollback()

query = """

select lower(replace(state, ' ', '_')) || '_' || lower(replace(state_name, ' ', '_')) || '_geojson.json'
from
(
    select distinct z.state, st.state_name
    from customers cu
         join zip_codes z
             on cu.zip = z.zip
         join states st
             on z.state = st.state
) as a
order by 1


"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

file_list = []

for row in rows:
    file_list.append(row[0])

print('GeoJSON Files:', file_list)


GeoJSON Files: ['ca_california_geojson.json', 'fl_florida_geojson.json', 'tn_tennessee_geojson.json', 'tx_texas_geojson.json', 'wa_washington_geojson.json']


In [29]:
customer_zip_geojson_file = open(os.path.join('geojson_data', 'customer_zip_geojson.json'), 'w')

customer_zip_geojson = {"type": "FeatureCollection", "features": []}

for file_name in file_list:
    
    directory_file_name = os.path.join('geojson_data', file_name)
    
    f = open(directory_file_name, 'r')
    geo = json.load(f)
    f.close()
    
    for fe in geo['features']:
        if fe['properties']['ZCTA5CE10'] in zip_list:
            customer_zip_geojson['features'].append(fe)
        
json.dump(customer_zip_geojson, customer_zip_geojson_file, indent=2)

customer_zip_geojson_file.close()
        
len(customer_zip_geojson['features'])

550